In [ ]:
# prepare sagemaker
from sagemaker.huggingface import HuggingFaceModel
from sagemaker import get_execution_role

model_dir = 's3://sagemaker-studio-147795258718-ywchj6yljj8/model.tar.gz'
endpoint_name = "news-classification-endpointv0001"

role = get_execution_role()
huggingface_model = HuggingFaceModel(
    model_data = model_dir,
    role = role,
    transformers_version = "4.49",
    pytorch_version = "2.6",
    py_version = "py312",
    entry_point = "deploy.py",
    # name='load-test-model',
    name='deployment-model',
)

def cleanup_sagemaker_endpoint(endpoint_name: str):
    sm = boto3.client("sagemaker")
    
    # delete endpoint (stops instance & billing)
    try:
        sm.delete_endpoint(EndpointName=endpoint_name)
        print(f"deleted endpoint: {endpoint_name}")
    except Exception as e:
        print(f"failed to delete endpoint {endpoint_name}: {e}")

    # delete endpoint config
    try:
        sm.delete_endpoint_config(EndpointConfigName=endpoint_name)
        print(f"deleted endpoint config: {endpoint_name}")
    except Exception as e:
        print(f"failed to delete endpoint config {endpoint_name}: {e}")

    # delete model
    try:
        sm.delete_model(ModelName=endpoint_name)
        print(f"deleted model: {endpoint_name}")
    except Exception as e:
        print(f"failed to delete model {endpoint_name}: {e}")

In [71]:
predictor = huggingface_model.deploy(
    initial_instance_count = 1,
    # instance_type = "ml.t3.medium",
    # instance_type = "ml.m5.xlarge",
    instance_type="ml.m5.large",
    endpoint_name=endpoint_name,
)

------!

In [73]:
# data = {"inputs": "Intel CFO says chipmaker has received $5.7 billion in outstanding CHIPS Act grants under Trump deal"}
data = {"inputs": "Nasdaq edges down as Nvidia falls on China market uncertainty"}

prediction = predictor.predict(data)
prediction

['{"predicted_label": "negative", "probabilities": [[0.19796720147132874, 0.02515009231865406, 0.7768826484680176]]}',
 'application/json']

In [74]:
cleanup_sagemaker_endpoint(endpoint_name)

deleted endpoint: news-classification-endpointv0001
deleted endpoint config: news-classification-endpointv0001
failed to delete model news-classification-endpointv0001: An error occurred (ValidationException) when calling the DeleteModel operation: Could not find model "news-classification-endpointv0001".
